# Liability Model II

**This notebook goes through creating a liability model

### IMPORT LIBRARIES  

In [ ]:
from AssetAllocation.datamanger import datamanger as dm
from AssetAllocation.analytics.liability_model import liabilityModel
import pandas as pd
from ipywidgets import interact, interact_manual, interactive

# IBT or Pension or Retirement
PLAN = 'IBT'

# % Probability of Sponsor Not Contributing Service Costs
UPS_CONTRB_PCT = .05

# Target Years to be "Fully Funded"
YRS_TO_FF = 5

# Target Fully Funded Ratio
FF_RATIO = 1.05

### IMPORT LIABILITY AND ASSET DATA
* Liability Cashflows:
    * PBO
    * Service Cost
* FTSE Curve
* Asset Data:
    * Market Value
    * Returns

In [ ]:
df_pbo_cfs = dm.get_cf_data('PBO')
df_sc_cfs = dm.get_cf_data('Service Cost')
df_ftse = dm.get_ftse_data()
plan_asset_data = dm.get_plan_asset_data()
plan_mv_cfs_dict = dm.get_plan_mv_cfs_dict()

### VIEW DATA

In [ ]:
data_dict = {'pbo cashflows': df_pbo_cfs, 'sc cashflows': df_sc_cfs,'ftse data': df_ftse, 
             'asset market values': plan_asset_data['mkt_value'],'asset return':plan_asset_data['return']}
@interact
def display_data(data = data_dict.keys()):
    return data_dict[data]

### TRANSFORM DATA TO LIABILITY MODEL INPUTS  

In [ ]:
pbo_cashflows = df_pbo_cfs[PLAN]
disc_factors = df_pbo_cfs['Time']
sc_cashflows = df_sc_cfs[PLAN]
liab_curve = dm.generate_liab_curve(df_ftse, pbo_cashflows)
asset_mv = dm.get_plan_asset_mv(plan_asset_data, PLAN)
asset_returns = dm.get_plan_asset_returns(plan_asset_data, PLAN)
liab_mv_cfs = dm.offset(plan_mv_cfs_dict[PLAN])

### CREATE LIABILITY MODEL

In [ ]:
liab_model = liabilityModel(pbo_cashflows, disc_factors, sc_cashflows, UPS_CONTRB_PCT,
                            asset_mv, liab_mv_cfs, asset_returns, liab_curve)

### VIEW LIABILITY MODEL DATA DICTIONARY

In [ ]:
@interact
def display_liab_data(data = liab_model.data_dict.keys()):
    return liab_model.data_dict[data]

### COMPUTE FULFILLMENT RETURN

In [ ]:
liab_model.compute_fulfill_ret(YRS_TO_FF, FF_RATIO)
liab_model.fulfill_irr

### COMPUTE FUTURE FUNDED STATUS

In [ ]:
liab_model.fulfill_solve(liab_model.fulfill_irr, YRS_TO_FF, FF_RATIO)

### EXCESS RETURN

In [ ]:
liab_model.excess_return

In [ ]:
liab_model.returns_ts.tail(12)